<a href="https://colab.research.google.com/github/DesmondChoy/create_adventure/blob/main/prompt_experimentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:
!pip install claudette -qqq

In [2]:
import os
import random
from claudette.core import Chat, Client, models
from google.colab import userdata

os.environ['ANTHROPIC_LOG'] = 'debug'
os.environ['ANTHROPIC_API_KEY'] = userdata.get('ANTHROPIC_API_KEY')




# Purpose: **Build a MVP**

This is a web-based "Choose your own adventure" RPG, but using LLMs and with an educational twist. Through the adventure, the reader is offered player agency choices (strategic decisions, unique equipment, companion, etc). These are closed-ended choices.

MVP: 5 pages.

How the workflow looks like in my mind:

# Intialization
- Pre-define educational trivia category. Next, begin by asking what type of adventure to embark on: choose_topic
- choose_topic = ['Magical Realms', 'Animal Adventures', 'Mystery and Detectives', 'Inventions and Gadgets']
- educational_trivia = random.choice(["Singapore history", "Farm animals", "Human body"])

# Page I & II
- A function start_adventure()  takes {choose_topic}  and {educational_trivia} as arguments
- start_adventure() produces Page I and II
- Page I sets the context and character development, with no choices. The plot/narrative must be related to {educational_trivia}
- Page II concludes with a Player agency choice. Answer is saved as user_choice

# Page III
- A function continue_adventure() takes two arguments, {story_so_far} and {user_choice}
- continue_adventure() continues to build the plot and is shaped by {user_choice}
- Page III concludes with a trivia question related to {educational_trivia}. Answer is saved as user_answer

# Page IV
- continue_adventure() is used again but it will need to take an additional argument {user_answer} together with {story_so_far}
- continue_adventure() continues to build the plot and is shaped by whether {user_answer} was correct or incorrect. If incorrect, there should be a lesson educating the reader on the correct answer.
- Page IV concludes with a Player agency choice. Answer is saved as user_choice

# Page V
- A function concludes adventure() takes two arguments, {story_so_far} and {user_choice}
- concludes adventure() reveals the plot and is shaped by {user_choice}. The plot is also related to {educational_trivia}.
- There will be a happy ending but there is also a hint of more adventures to come.

# Some comments
- Persistence: The story state between pages is saved as string variables for now. I'll think of a better solution later down the road.

- Modularity: The focus is on shipping a MVP and refactoring will be done later. I'm open to simple, intuitive suggestions on how the functions should like like e.g. class structures

- Story generation: Claude Sonnet 3.5 will be used to generate the narrative dynamically.



To be included later:
- Difficulty level (easy/medium/hard) - this will define maths qns.
- Rendered images



# Intialization

In [3]:
# Pre-define educational trivia categories and adventure types
choose_topic = ['Magical Realms', 'Animal Adventures', 'Mystery and Detectives', 'Inventions and Gadgets']
educational_trivia = ["Singapore history", "Farm animals", "Human body"]


In [4]:
client = Client(models[1]) #claude-3-5-sonnet-20240620
# Initialize the Chat client
chat = Chat(cli=client)

In [5]:
# system_prompt = """
# You are now a captivating children's author and storyteller, crafting second-person narratives that whisk young readers into thrilling adventures.
# Your audience is children and teenagers, and your stories burst with vivid imagery, clever humor, and whimsical characters.
# You use age-appropriate language, sprinkle in interactive elements, and weave subtle positive messages throughout your tales.
# Your narratives span diverse, imaginative settings and employ playful language to delight your audience.
# With enthusiasm and creativity, you aim to spark joy, laughter, and a love for storytelling in every interaction,
# using cliffhangers and suspense to keep young minds engaged and eager for more.
# """

# Pages

In [6]:
# Function to start the adventure
def start_adventure(topic, trivia):
    prompt = f"""
    Create the first two pages of an interactive educational adventure for children.
    The adventure should be in the theme of {topic} and incorporate facts about {trivia}.

    Page I: Set the context and introduce the main character. No choices should be presented.

    Page II: Continue the story and end with a choice for the reader.
    Present two clear options for the reader to choose from.

    Format the output as follows:
    PAGE I: [Content for Page I]
    PAGE II: [Content for Page II]
    CHOICE A: [First option]
    CHOICE B: [Second option]
    """

    response = chat(prompt, temp=0.8)
    return response.content[0].text

# Play

In [7]:
# Function to start the adventure
def start_adventure(topic, trivia):
    prompt = f"""
    Create the first two pages of an interactive educational adventure for children.
    The adventure should be in the theme of {topic} and incorporate facts about {trivia}.

    Page I: Set the context and introduce the main character. No choices should be presented.

    Page II: Continue the story and end with a choice for the reader.
    Present two clear options for the reader to choose from.

    Format the output as follows:
    PAGE I: [Content for Page I]
    PAGE II: [Content for Page II]
    CHOICE A: [First option]
    CHOICE B: [Second option]
    """

    response = ""
    for chunk in chat(prompt, stream=True, temp=0.8):
        response += chunk
        print(chunk, end='', flush=True)
    return response

# Function to continue the adventure
def continue_adventure(story_so_far, user_choice, trivia, is_trivia_page=False):
    if is_trivia_page:
        prompt = f"""
        Continue the interactive educational adventure. Use the following context:

        Story so far: {story_so_far}
        User's last choice: {user_choice}

        Create the next page of the story, incorporating the user's choice.
        At the end of this page, include a trivia question related to {trivia}.
        The question should have two possible answers, A and B, with only one being correct.

        Format the output as follows:
        PAGE CONTENT: [Content for the page]
        TRIVIA QUESTION: [Trivia question]
        CHOICE A: [First answer option]
        CHOICE B: [Second answer option]
        CORRECT: [A or B]
        """
    else:
        prompt = f"""
        Continue the interactive educational adventure. Use the following context:

        Story so far: {story_so_far}
        User's last choice: {user_choice}

        Create the next page of the story, incorporating the user's choice.
        End this page with another choice for the reader.

        Format the output as follows:
        PAGE CONTENT: [Content for the page]
        CHOICE A: [First option]
        CHOICE B: [Second option]
        """

    response = ""
    for chunk in chat(prompt, stream=True, temp=0.8):
        response += chunk
        print(chunk, end='', flush=True)
    return response

# Function to conclude the adventure
def conclude_adventure(story_so_far, user_choice, trivia):
    prompt = f"""
    Conclude the interactive educational adventure. Use the following context:

    Story so far: {story_so_far}
    User's last choice: {user_choice}

    Create the final page of the story, incorporating the user's choice and facts about {trivia}.
    The ending should be happy but hint at more adventures to come.

    Format the output as follows:
    CONCLUSION: [Content for the conclusion]
    """

    response = ""
    for chunk in chat(prompt, stream=True, temp=0.8):
        response += chunk
        print(chunk, end='', flush=True)
    return response

# Function to extract choices from the response
def extract_choices(response):
    lines = response.split('\n')
    choices = {}
    for line in lines:
        if line.startswith('CHOICE A:'):
            choices['A'] = line.split('CHOICE A:')[1].strip()
        elif line.startswith('CHOICE B:'):
            choices['B'] = line.split('CHOICE B:')[1].strip()
    return choices

# Function to extract trivia question and correct answer
def extract_trivia(response):
    lines = response.split('\n')
    trivia = {}
    for line in lines:
        if line.startswith('TRIVIA QUESTION:'):
            trivia['question'] = line.split('TRIVIA QUESTION:')[1].strip()
        elif line.startswith('CHOICE A:'):
            trivia['A'] = line.split('CHOICE A:')[1].strip()
        elif line.startswith('CHOICE B:'):
            trivia['B'] = line.split('CHOICE B:')[1].strip()
        elif line.startswith('CORRECT:'):
            trivia['correct'] = line.split('CORRECT:')[1].strip()
    return trivia

In [8]:
# Main game loop
def play_game():
    topic = random.choice(choose_topic)
    trivia = random.choice(educational_trivia)

    print(f"Welcome to your {topic} adventure, where you'll learn about {trivia}!")

    # Start the adventure
    story = start_adventure(topic, trivia)
    choices = extract_choices(story)

    # Get user's choice
    user_choice = input("Enter your choice (A or B): ").upper()
    while user_choice not in ['A', 'B']:
        user_choice = input("Invalid choice. Please enter A or B: ").upper()

    # Continue the adventure (Page III with trivia)
    story += f"\nUser chose: {user_choice}"
    next_page = continue_adventure(story, user_choice, trivia, is_trivia_page=True)
    trivia_info = extract_trivia(next_page)

    print(f"\nTrivia Question: {trivia_info['question']}")
    print(f"A: {trivia_info['A']}")
    print(f"B: {trivia_info['B']}")

    # Get user's answer to trivia
    user_answer = input("Enter your answer (A or B): ").upper()
    while user_answer not in ['A', 'B']:
        user_answer = input("Invalid choice. Please enter A or B: ").upper()

    # Continue the adventure (Page IV)
    story += f"\n{next_page}\nUser's trivia answer: {user_answer}"
    next_page = continue_adventure(story, user_choice, trivia)
    choices = extract_choices(next_page)

    # Get user's final choice
    user_choice = input("Enter your choice (A or B): ").upper()
    while user_choice not in ['A', 'B']:
        user_choice = input("Invalid choice. Please enter A or B: ").upper()

    # Conclude the adventure
    story += f"\n{next_page}\nUser chose: {user_choice}"
    conclusion = conclude_adventure(story, user_choice, trivia)
    print(conclusion)

In [9]:
# Run the game
play_game()

Welcome to your Inventions and Gadgets adventure, where you'll learn about Human body!
PAGE I:
Meet Zoe, a curious 10-year-old with wild curly hair and a passion for inventing gadgets. One day, while tinkering in her backyard workshop, Zoe accidentally activated her latest creation: the Bodyscope 3000. With a flash of light and a puff of smoke, Zoe found herself shrunk to the size of a dust speck and transported inside a giant human body! As she marveled at the enormous cells around her, Zoe realized this was her chance to explore the human body like never before and gather inspiration for her next big invention.

PAGE II:
Zoe floated through a vast network of blood vessels, her eyes wide with wonder. She watched red blood cells zoom past her, carrying oxygen to different parts of the body. Suddenly, she felt a strong current pulling her towards a massive, rhythmically beating structure. "That must be the heart!" Zoe exclaimed, recalling that it pumps about 2,000 gallons of blood each 